In [6]:
import pandas as pd
import string
import geonamescache
import re
import nltk
from nltk.corpus import stopwords
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [7]:

tw_mitaka_a = pd.read_csv("tw_mitaka_814.csv")
tw_mitaka_1 = pd.read_csv("tw_mitaka1.csv")
x = pd.concat([tw_mitaka_a, tw_mitaka_1]).drop_duplicates()
x.to_csv('tw_mitaka1.csv', index=False)


In [8]:
tw_mitaka = pd.read_csv("tw_mitaka1.csv")
tw_mitaka['created_at'] =  pd.to_datetime(tw_mitaka['created_at'])

In [9]:
tw_mitaka.shape

(1727, 7)

In [10]:
tw_mitaka.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1727 entries, 0 to 1726
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype              
---  ------      --------------  -----              
 0   tweet_id    1727 non-null   int64              
 1   username    1727 non-null   object             
 2   text        1727 non-null   object             
 3   created_at  1727 non-null   datetime64[ns, UTC]
 4   location    1111 non-null   object             
 5   source      1727 non-null   object             
 6   url         1727 non-null   object             
dtypes: datetime64[ns, UTC](1), int64(1), object(5)
memory usage: 94.6+ KB


In [11]:
tw_mitaka.head()

,tweet_id,username,text,created_at,location,source,url
0,1559563130081910784,railgunreformed,i will protect Asa Mitaka with my life,2022-08-16 15:30:00+00:00,she/her • LgbT • 20 • ASD1-ADD,Twitter Web App,https://twitter.com/railgunreformed/status/155...
1,1559561877755396098,yoruzuya_,Asa Mitaka nice decision there she's not getti...,2022-08-16 15:25:01+00:00,NaN,Twitter for Android,https://twitter.com/yoruzuya_/status/155956187...
2,1559561459897700352,MonkeyDJosh_,Mitaka is her I knew it from the first moment ...,2022-08-16 15:23:21+00:00,NaN,Twitter for iPhone,https://twitter.com/MonkeyDJosh_/status/155956...
3,1559560601172447233,Baleygr086,After all the hype surrounding Denji's comebac...,2022-08-16 15:19:57+00:00,CA,Twitter Web App,https://twitter.com/Baleygr086/status/15595606...
4,1559560258250362880,exstnctionis,https://t.co/lvUEcCkC71 asa mitaka is clum...,2022-08-16 15:18:35+00:00,NaN,Twitter for iPhone,https://twitter.com/exstnctionis/status/155956...


In [ ]:
#Stop words, links, emotes, non-ascii characters are removed.
stops = set(stopwords.words('english'))

for i in range(len(tw_mitaka['text'])):
    tweet_text = []
    #https://stackoverflow.com/a/24399874
    tw_mitaka['text'][i] = re.sub(r'[^\x00-\x7f]',r'', tw_mitaka['text'][i])
    #https://stackoverflow.com/a/65330690
    tw_mitaka['text'][i] = re.sub(r"http\S+", "", tw_mitaka['text'][i])
    
    words = tw_mitaka['text'][i].lower().split()
    words = [''.join(letter for letter in word if letter not in (string.punctuation)) for word in words]
    for word in words:
        if word not in stops:
            tweet_text.append(word)

    tw_mitaka['text'].iloc[i] = ' '.join(tweet_text)

In [13]:
print(tw_mitaka['text'][25])

shes wont trust mitaka


In [14]:
#Word frequency dict
word_freq = {}

for text_list in tw_mitaka['text']:
    for word in text_list.split():
        if word not in word_freq:
            word_freq[word] = 0
        word_freq[word] += 1

In [15]:
dict(list(word_freq.items())[0:5])

{'protect': 1, 'asa': 365, 'mitaka': 791, 'life': 15, 'nice': 10}

In [16]:
#Creating dataframe from word freq dict
df_wordfreq = pd.DataFrame.from_dict(word_freq, orient='index', columns=['Freq'])

In [17]:
df_wordfreq.sort_values('Freq', ascending=False).head(5)

,Freq
devil,863
war,852
mitaka,791
chainsawman,443
asa,365


#### VADER Sentimental Analysis

In [18]:
#Sentimental analysis function with VADER
def sentiment_scores(sentence):
    sid_obj = SentimentIntensityAnalyzer()
    sentiment_dict = sid_obj.polarity_scores(sentence)
    if sentiment_dict['compound'] >= 0.05 :
        return("Positive")
    elif sentiment_dict['compound'] <= -0.05 :
        return("Negative")
    else :
        return("Neutral")

In [19]:
#Adding score column for sentiment analysis
tw_mitaka_vader = tw_mitaka.copy()

In [20]:
#VADER Sentimental analysis on text. 
score_list = []
for i in tw_mitaka_vader['text']:
    score = sentiment_scores(i)
    score_list.append(score)

In [21]:
tw_mitaka_vader['score'] = score_list

In [22]:
tw_mitaka_vader.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1727 entries, 0 to 1726
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype              
---  ------      --------------  -----              
 0   tweet_id    1727 non-null   int64              
 1   username    1727 non-null   object             
 2   text        1727 non-null   object             
 3   created_at  1727 non-null   datetime64[ns, UTC]
 4   location    1111 non-null   object             
 5   source      1727 non-null   object             
 6   url         1727 non-null   object             
 7   score       1727 non-null   object             
dtypes: datetime64[ns, UTC](1), int64(1), object(6)
memory usage: 108.1+ KB


### Can see that most tweets are "negative" according to the algorithm, but I want to say that's mostly due to the language of the queried keywords in the first place.

In [23]:
print(tw_mitaka_vader.groupby('score').size())

score
Negative    973
Neutral     412
Positive    342
dtype: int64


In [24]:
print(tw_mitaka.groupby('username').size().sort_values(ascending=False)[0:10])

username
retromanjapan     81
soubusen7         21
Rod_anderis       21
loveabyssal       20
dnbooru           10
hyegumi           10
lonehuman123      10
rainyxink          9
AirRapid_HE433     9
SassyGayHux        8
dtype: int64


In [95]:
print(tw_mitaka.groupby('location').size().sort_values(ascending=False)[0:10])

location
Tokyo, Japan                      84
千葉～三鷹                             21
ic: mazamuno → qt count: 49       20
she/her                           18
Tokyo                             14
dying non human innocent Earth    10
aroace - 23 - rey                  9
❮SY13❯舞子公園(兵庫県 神戸市)                9
Tokyo-to, Japan                    8
Malaysia                           7
dtype: int64


In [26]:
print(tw_mitaka.groupby('source').size().sort_values(ascending=False)[0:5])

source
Twitter for Android    537
Twitter for iPhone     523
Twitter Web App        413
Revive Social App       81
Twitter for iPad        49
dtype: int64


In [27]:
tw_mitaka['location'].nunique()

725

In [90]:
loc_freq = {}
for i in tw_mitaka['location']:
    i = str(i).lower()
    if i == 'nan':
        pass
    else:
        location_split = i.split()
        for word in location_split:
            if word not in loc_freq:
                loc_freq[word] = 0
            loc_freq[word]+=1
#Have to remove puncation from each word

In [92]:
loc_df = pd.DataFrame(loc_freq.values(), index = loc_freq.keys(), columns=['Frequency'])

In [94]:
loc_df.sort_values(by = 'Frequency', ascending=False)[0:5]

,Frequency
japan,98
she/her,95
"tokyo,",84
|,73
20,35


In [ ]:
gc = geonamescache.GeonamesCache()
countries = gc.get_countries()
cities = gc.get_cities()
states = gc.get_us_states()

#Countries
#for v in countries.values():
#    print(v['iso'], v['name'], v['capital'])

#Cities
'''
for city in cities.values():
    print(city['name'])
    for altnames in city['alternatenames']:
        print(altnames)
'''

#US States
'''
for k, v in states.items():
    print(v['code'], v['name'])
'''

In [ ]:
'''
Next Steps:
Create visualizations of findings.
Time, Source, Score, username maybe (who's tweeting the most?), location (need to clean)

Visualizations:

Word Frequency.
VADER Sentimental Analysis - # of Positives and # negatives
	Provide an example tweet for each.

Username frequency
Location frequency 
Source frequency
Time of tweets (using SQL is best) - Most active time?
'''